# Hosts/Managers: Working with trait versions

This notebook illustrates how a manager and host can work together despite using different trait versions.


## Versioned traits

### Schema subpackages

Trait packages generated by `traitgen` can include subpackages for all available schema versions, with the top-level namespace containing aliases to the most-recent available version.

To illustrate this, we use the versioned trait mockups under `resources/working_with_trait_versions`. 


In [1]:
from resources.working_with_trait_versions.openassetio_example import traits, specifications
from resources.working_with_trait_versions.openassetio_example import v1
from resources.working_with_trait_versions.openassetio_example import v2


assert traits is v2.traits
assert specifications is v2.specifications

assert v1.traits is not v2.traits
assert v1.traits is not v2.specifications

Host applications that bundle the trait package may use the non-versioned top-level package by default. 

For Python manager plugins, if 

* the host application's largest (most-recent) schema version does not match the schema version the Python plugin was developed against; and
* the Python plugin uses the default (top-level) traits/specifications packages

the application may hit unexpected runtime errors. This is because the structure of trait/specification view classes may be different, or simply not exist.

A Python manager plugin should therefore use a versioned namespace. This solves the problem when the plugin is loaded into an application that defaults to a _higher_ schema version than the plugin was developed for. 

However, if the host application's largest available schema version is too _low_ for the manager plugin, then using a versioned namespace would suffer an `ImportError` when attempting to `import` the (too new) versioned namespace. This can either be left to fail (better to fail early), or be tolerated within the plugin by catching the exception and falling back to an older version.

C++ plugins have the trait view classes (privately) compiled into them, so do not depend on the schema version that the host application was built against. Incompatibilities only become apparent at runtime, when incoming trait data is found to be of a version unsupported by the manager or host.

### Trait views within subpackages

We imagine an industry where there are only 4 traits, `AddedTrait`, `RemovedTrait`, `UnchangedTrait` and `UpdatedTrait`, which are used across entities, relationships, policies and locales. The traits themselves have no meaning, they are named purely to give a hint as to how they change in subsequent trait schema versions.

Within a subpackage, all traits within that subpackage's schema version are included, with unchanged traits being duplicated. Duplication may be replaced with aliasing in a future update of `traitgen`.

The current and any older versions of a trait are also represented under a subpackage, with a version suffix on the class name. The class name without a version suffix is an alias to the latest version available under that schema version.

Traits can be removed from a schema, but will still exist under the schema subpackage if the package was generated to include older schema versions.

In [10]:
assert v2.traits.example.UnchangedTrait is not v1.traits.example.UnchangedTrait
assert v2.traits.example.UpdatedTrait is not v1.traits.example.UpdatedTrait

assert v2.traits.example.UpdatedTrait is v2.traits.example.UpdatedTrait_v2
assert v2.traits.example.UpdatedTrait is not v2.traits.example.UpdatedTrait_v1
assert "UpdatedTrait_v2" not in v1.traits.example.__dict__

# RemovedTrait is missing from v2, but is still added to the v2
# subpackage to aid backward compatibility.
assert "RemovedTrait" in v2.traits.example.__dict__
assert "RemovedTrait" in v1.traits.example.__dict__

assert "AddedTrait" in v2.traits.example.__dict__
assert "AddedTrait" not in v1.traits.example.__dict__

Each trait encodes its version in its unique ID. 

The schema version is bumped whenever one or more traits or specifications are added/removed/updated, forming the next schema version. This means the maximum possible version of a trait is bounded by the top-level schema version (inclusive). Adding a new trait will result in a schema version bump, but the new trait itself will start at version 1.

In [3]:
# An updated trait has a different ID in each subpackage.
assert v1.traits.example.UpdatedTrait.kId == "openassetio-example:example.Updated.v1"
assert v2.traits.example.UpdatedTrait.kId == "openassetio-example:example.Updated.v2"

# A newly added trait starts at version 1, despite the schema
# (subpackage) version being greater than 1.
assert v2.traits.example.AddedTrait.kId == "openassetio-example:example.Added.v1"

This means trait view classes from one schema version cannot be used to read traits of another version, unless the trait is unchanged between schema versions.

In [4]:
from openassetio.trait import TraitsData


v1_data, v2_data, unchanged_data = TraitsData(), TraitsData(), TraitsData()

v1.traits.example.UpdatedTrait.imbueTo(v1_data)
v2.traits.example.UpdatedTrait.imbueTo(v2_data)
v1.traits.example.UnchangedTrait.imbueTo(unchanged_data)

assert v1.traits.example.UpdatedTrait.isImbuedTo(v1_data) is True
assert v1.traits.example.UpdatedTrait.isImbuedTo(v2_data) is False

assert v2.traits.example.UpdatedTrait.isImbuedTo(v1_data) is False
assert v2.traits.example.UpdatedTrait.isImbuedTo(v2_data) is True

assert v1.traits.example.UnchangedTrait.isImbuedTo(unchanged_data) is True
assert v2.traits.example.UnchangedTrait.isImbuedTo(unchanged_data) is True

However, a workaround to this is to use the fallback versioned trait view classes in the newer schema subpackage

In [ ]:
assert v2.traits.example.UpdatedTrait_v1.isImbuedTo(v1_data) is True
assert v2.traits.example.UpdatedTrait_v2.isImbuedTo(v2_data) is True

### Specifications

Specifications are a way to document well-known sets of traits that categorize entities, relationships, locales, or policies. Agreement on these as an industry is crucial for effective interop.

However, they do not have an independent version - their version is implicit in the (versioned) traits that they compose, and in the overall schema version where they are defined. A major consequence of this is that no specification version is embedded in the data itself.

If the raison d'être of specifications is a way to have an industry-standard collection of trait sets, then is the exact version of those traits really important? On the assumption that a new version of a trait doesn't fundamentally change its meaning (otherwise it would be an entirely new trait), then it's reasonable to say that specifications are trait version agnostic.

So specifications are invaluable as documentation of common trait sets. However, the auto-generateed `Specification` view classes should be used with caution: using a specification view class to categorize an entity may lead to unexpected false negatives when the trait versions do not line up.


In [5]:
entity_data = TraitsData(
    {v1.traits.example.UnchangedTrait.kId, v1.traits.example.UpdatedTrait.kId})

# As long as the trait set of the specification lines up with the
# incoming data, we can use it to categorize an entity.
is_an_example_entity = v1.specifications.example.ExampleSpecification.kTraitSet.issubset(
    entity_data.traitSet())
assert is_an_example_entity is True

# Conceptually, the entity is still an Example, but subsequent updates
# to the specification mean the version suffix on some trait IDs have
# updated and no longer match the incoming data, so we get a false
# negative.
is_an_example_entity = v2.specifications.example.ExampleSpecification.kTraitSet.issubset(
    entity_data.traitSet())
assert is_an_example_entity is False


## Example 

The following sections will define a manager and a host and explore how they can communicate despite no prior knowledge of what trait versions each side will send.

### Prerequisites

First we must define some boilerplate.

In [6]:
from openassetio.hostApi import HostInterface
from openassetio.managerApi import Host, HostSession
from openassetio.log import ConsoleLogger, SeverityFilter


class NotebookHostInterface(HostInterface):
    def identifier(self):
        return "org.jupyter.notebook"

    def displayName(self):
        return "Jupyter Notebook"


host_session = HostSession(Host(NotebookHostInterface()), SeverityFilter(ConsoleLogger()))

### Manager 

In the following a manager implementation is defined that relies solely on the versioned trait mockups under `resources/working_with_trait_versions`. 

The idea is simply to tease out possible patterns of versioned trait combinations and access patterns that might cause problems for implementors. As such, the semantics are nonsense, but hopefully the branching logic is roughly representative.

In [11]:
from openassetio.managerApi import ManagerInterface, EntityReferencePagerInterface
from openassetio import errors, access, EntityReference

from resources.working_with_trait_versions.openassetio_example.v2 import traits

an_entity_ref_str = "example://entity"


class ExampleManagerInterface(ManagerInterface):

    def identifier(self):
        return "org.openassetio.example.manager"

    def displayName(self):
        return "Example Manager"

    def hasCapability(self, capability):
        return capability in (
            ManagerInterface.Capability.kEntityReferenceIdentification,
            ManagerInterface.Capability.kManagementPolicyQueries,
            ManagerInterface.Capability.kEntityTraitIntrospection,
            ManagerInterface.Capability.kResolution,
            ManagerInterface.Capability.kPublishing,
            ManagerInterface.Capability.kRelationshipQueries,
            ManagerInterface.Capability.kDefaultEntityReferences,
        )

    def isEntityReferenceString(self, someString, _hostSession):
        return someString.startswith("example://")

    def managementPolicy(self, traitSets, policyAccess, context, _hostSession):
        # Initialise default empty response, to be filled in below.
        policy_datas = [TraitsData() for _ in traitSets]

        # We care about the specific Context locale under which this
        # query was made.
        is_locale_special = False
        special_locale_value = False

        # Assume we know from reading release notes that UnchangedTrait
        # hasn't changed, so we can use v2 knowing that v1 is equivalent.
        if traits.example.UnchangedTrait.isImbuedTo(context.locale):
            # UpdatedTrait changes between versions, but we know from
            # release notes that the property we're interested in still
            # exists semantically, it's just the name has changed.

            # Check if v2 of UpdatedTrait is imbued, and if so extract
            # the property via its new name.
            if traits.example.UpdatedTrait_v2.isImbuedTo(context.locale):
                is_locale_special = True
                special_locale_value = v2.traits.example.UpdatedTrait(
                    context.locale).getPropertyThatWasRenamed(defaultValue=special_locale_value)

            # If v2 of UpdatedTrait was not imbued, fall back to v1. If
            # imbued, extract the property via its old name. If both v2
            # and v1 were imbued for some reason, prefer v2.
            elif traits.example.UpdatedTrait_v1.isImbuedTo(context.locale):
                is_locale_special = True
                special_locale_value = traits.example.UpdatedTrait_v1(
                    context.locale).getPropertyToRename(defaultValue=special_locale_value)

        for trait_set, policy_data in zip(traitSets, policy_datas):
            if policyAccess is access.PolicyAccess.kRead:

                # Read is only supported when the locale is "special".
                if not is_locale_special:
                    continue

                # Only sets with the v2 Added or v1 Removed trait are
                # supported.
                if not (traits.example.RemovedTrait.kId in trait_set or
                        traits.example.AddedTrait.kId in trait_set):
                    continue

                if special_locale_value is True:
                    # Since the locale's "special" value is set, we are
                    # capable of providing property values for either or
                    # both v1 and v2 of UpdatedTrait simultaneously.

                    if traits.example.UpdatedTrait_v2.kId in trait_set:
                        traits.example.UpdatedTrait_v2.imbueTo(policy_data)

                    if traits.example.UpdatedTrait_v1.kId in trait_set:
                        traits.example.UpdatedTrait_v1.imbueTo(policy_data)
                else:
                    # Since the locale's "special" value is not set, we
                    # can only provide property values for either v1 or
                    # v2, but not both, of UpdatedTrait. We prefer v2.

                    if (traits.example.UpdatedTrait_v2.kId in trait_set
                            and not traits.example.UpdatedTrait_v1.kId in trait_set):
                        traits.example.UpdatedTrait_v2.imbueTo(policy_data)
                    elif traits.example.UpdatedTrait_v1.kId in trait_set:
                        traits.example.UpdatedTrait_v1.imbueTo(policy_data)

                continue
            else:
                # Other access modes all the same.
                if not is_locale_special:
                    continue

                # Prefer v2, ignoring v1 if it is set.

                if traits.example.UpdatedTrait_v2.kId in trait_set:
                    traits.example.UpdatedTrait_v2.imbueTo(policy_data)
                elif traits.example.UpdatedTrait_v1.kId in trait_set:
                    traits.example.UpdatedTrait_v1.imbueTo(policy_data)

        return policy_datas

    def defaultEntityReference(
            self, traitSets, defaultEntityAccess, context, hostSession, successCallback,
            errorCallback):
        for idx, trait_set in enumerate(traitSets):
            is_an_unchanged = traits.example.UnchangedTrait.kId in trait_set
            is_an_updated = (traits.example.UpdatedTrait_v2.kId in trait_set or
                             traits.example.UpdatedTrait_v1.kId in trait_set)
            is_a_removed = traits.example.RemovedTrait.kId in trait_set
            is_an_added = traits.example.AddedTrait.kId in trait_set

            if is_an_unchanged and is_an_updated and is_a_removed:
                # Only possible with v1 schema/trait
                entity_ref = "example://default/removed"
            elif is_an_unchanged and is_an_updated and is_an_added:
                # Only possible with v2 schema/trait
                entity_ref = "example://default/added"
            elif is_an_unchanged and is_an_updated:
                # v1 or v2 schemas
                entity_ref = "example://default"
            else:
                # Any other unrecognized trait set
                errorCallback(
                    idx, errors.BatchElementError(
                        errors.BatchElementError.ErrorCode.kInvalidTraitSet,
                        "Entity trait set unrecognised"))
                continue

            # Not really important here, but for completeness handle all
            # access modes.
            if defaultEntityAccess is access.DefaultEntityAccess.kWrite:
                entity_ref += "/new"
            elif defaultEntityAccess is access.DefaultEntityAccess.kCreateRelated:
                entity_ref += "/child/new"

            successCallback(idx, entity_ref)

    def entityTraits(
            self,
            entityRefs,
            entityTraitsAccess,
            context,
            hostSession,
            successCallback,
            errorCallback):
        for idx, entity_ref in enumerate(entityRefs):
            # This manager only supports one entity ref.
            if str(entity_ref) == an_entity_ref_str:
                # Read access
                if entityTraitsAccess == access.EntityTraitsAccess.kRead:
                    #  We use the ExampleSpecification - a well-known
                    #  trait set with all traits required to categorize
                    #  an entity as an Example. No way to know what
                    #  version the host would prefer. So prefer latest,
                    #  v2.
                    successCallback(idx, v2.specifications.example.ExampleSpecification.kTraitSet)
                else:
                    # Minimum required for publishing is a reduced set.
                    successCallback(
                        idx, v2.specifications.example.ExampleSpecification.kTraitSet - {
                            traits.example.AddedTrait.kId})

            else:
                errorCallback(
                    idx, errors.BatchElementError(
                        errors.BatchElementError.ErrorCode.kEntityResolutionError,
                        "Entity doesn't exist"))

    def resolve(
            self,
            entityRefs,
            traitSet,
            resolveAccess,
            context,
            hostSession,
            successCallback,
            errorCallback):
        traits_datas = [TraitsData() for _ in entityRefs]

        for idx, (entity_ref, traits_data) in enumerate(zip(entityRefs, traits_datas)):
            if str(entity_ref) == an_entity_ref_str:
                if resolveAccess == access.ResolveAccess.kRead:
                    # Support either v1 or v2 (or both) for read.
                    if traits.example.UpdatedTrait_v2.kId in traitSet:
                        trait = traits.example.UpdatedTrait_v2(traits_data)
                        trait.setPropertyToKeep("value")
                        trait.setPropertyThatWasRenamed(True)
                        trait.setPropertyThatWasAdded(123.456)
                    if traits.example.UpdatedTrait_v1.kId in traitSet:
                        trait = traits.example.UpdatedTrait_v1(traits_data)
                        trait.setPropertyToKeep("value")
                        trait.setPropertyToRename(True)
                        trait.setPropertyToRemove(False)

                elif resolveAccess == access.ResolveAccess.kManagerDriven:
                    # Only support v2 for publishing workflows.
                    if traits.example.UpdatedTrait_v2.kId in traitSet:
                        trait = traits.example.UpdatedTrait_v1(traits_data)
                        trait.setPropertyThatWasAdded(456.789)
            else:
                # Similar for other entities.
                if resolveAccess == access.ResolveAccess.kManagerDriven:
                    # Only support latest version (v2) for publishing workflows.
                    if traits.example.UpdatedTrait.kId in traitSet:
                        trait = traits.example.UpdatedTrait(traits_data)
                        trait.setPropertyThatWasAdded(789.123)
                        trait.setPropertyThatWasRenamed(True)

            successCallback(idx, traits_data)

    def getWithRelationship(
            self,
            entityRefs,
            relationshipTraitsData,
            resultTraitSet,
            pageSize,
            relationsAccess,
            context,
            hostSession,
            successCallback,
            errorCallback):

        # Parse out important aspects of the type of relationship.

        is_rel_unchanged = traits.example.UnchangedTrait.isimbuedTo(
            relationshipTraitsData)

        rel_v2_updated = traits.example.UpdatedTrait_v2(relationshipTraitsData)
        rel_v1_updated = traits.example.UpdatedTrait_v1(relationshipTraitsData)
        is_rel_updated = rel_v2_updated.isImbued() or rel_v1_updated.isImbued()

        important_property = rel_v2_updated.getPropertyThatWasRenamed(
            defaultValue=rel_v1_updated.getPropertyToRename(defaultValue=False))

        is_a_child_relationship = is_rel_unchanged and is_rel_updated

        # Parse out important aspects of the type of expected result entity.

        result_type = "none"
        if {traits.example.AddedTrait.kId,
            traits.example.UnchangedTrait.kId}.issubset(resultTraitSet):
            result_type = "component"
        elif {traits.example.RemovedTrait.kId,
              traits.example.UnchangedTrait.kId}.issubset(resultTraitSet):
            result_type = "element"

        # Loop through input entities.

        for idx, entity_ref in enumerate(entityRefs):

            rels = []

            if relationsAccess is access.RelationsAccess.kRead:
                if str(entity_ref) == an_entity_ref_str:
                    if is_a_child_relationship:
                        if result_type == "component":
                            rels.append(EntityReference("example://entity/component/1"))
                            rels.append(EntityReference("example://entity/component/2"))

                        elif result_type == "element":
                            rels.append(EntityReference("example://entity/element/a"))

                        if result_type != "none" and important_property is True:
                            rels.append(EntityReference("example://entity/component/3/element/b"))
                else:
                    # Similar for other entity refs.
                    ...

            elif relationsAccess is access.RelationsAccess.kWrite:
                if str(entity_ref) == an_entity_ref_str:
                    if is_a_child_relationship:
                        # Only respond for "component" (i.e. v2) relationships.
                        if result_type == "component":
                            rels.append(EntityReference("example://entity/component/1/edit"))
                            rels.append(EntityReference("example://entity/component/2/edit"))
                else:
                    # Similar for other entity refs.
                    ...

            elif relationsAccess is access.RelationsAccess.kCreateRelated:
                if str(entity_ref) == an_entity_ref_str:
                    if is_a_child_relationship:
                        # Only respond for "component" (i.e. v2) relationships.
                        if result_type == "component":
                            rels.append(EntityReference("example://entity/component/new"))
                else:
                    # Similar for other entity refs.
                    ...

            successCallback(idx, ExampleEntityReferencePagerInterface(pageSize, rels))

    def getWithRelationships(
            self,
            entityReference,
            relationshipTraitsDatas,
            resultTraitSet,
            pageSize,
            relationsAccess,
            context,
            hostSession,
            successCallback,
            errorCallback):
        # Largely same as getWithRelationship, with outer loop changed.
        ...

    def preflight(
            self,
            targetEntityRefs,
            traitsDatas,
            publishingAccess,
            context,
            hostSession,
            successCallback,
            errorCallback):

        for idx, (entity_ref, traits_data) in enumerate(zip(targetEntityRefs, traitsDatas)):
            if str(entity_ref) != an_entity_ref_str:
                errorCallback(
                    idx, errors.BatchElementError(
                        errors.BatchElementError.ErrorCode.kEntityAccessError,
                        "Cannot publish to this entity"))
                continue

            # Categorise the data to publish
            is_an_unchanged = traits.example.UnchangedTrait.isImbuedTo(traits_data)
            is_an_updated = (traits.example.UpdatedTrait_v2.isImbuedTo(traits_data) or
                             traits.example.UpdatedTrait_v1.isImbuedTo(traits_data))
            is_a_removed = traits.example.RemovedTrait.isImbuedTo(traits_data)
            is_an_added = traits.example.AddedTrait.isImbuedTo(traits_data)

            # Based on the given traits, categorize to pipeline-specific, "type"
            is_an_item = is_an_added and is_an_updated
            is_a_unit = is_a_removed and is_an_updated
            is_an_ingredient = is_an_unchanged and is_an_updated

            # At least and only one, i.e. n-ary xor.
            if int(is_an_item) + int(is_a_unit) + int(is_an_ingredient) != 1:
                errorCallback(
                    idx, errors.BatchElementError(
                        errors.BatchElementError.ErrorCode.kInvalidPreflightHint,
                        "Unsupported traits for publishing to this entity"))
                continue

            if is_an_item:
                v2_updated_trait = traits.example.UpdatedTrait_v2(traits_data)
                v1_updated_trait = traits.example.UpdatedTrait_v1(traits_data)

                if v2_updated_trait.isImbued():
                    specialisation = v2_updated_trait.getPropertyToKeep()
                else:  # at this point guaranteed that v1 is imbued.
                    specialisation = v1_updated_trait.getPropertyToKeep()

                successCallback(
                    idx, EntityReference(
                        f"example://working_ref/item/{specialisation}/new"))

            elif is_a_unit:
                successCallback(idx, EntityReference(f"example://working_ref/unit/new"))

            elif is_an_ingredient:
                successCallback(idx, EntityReference(f"example://working_ref/ingredient/new"))

    def register(
            self,
            targetEntityRefs,
            entityTraitsDatas,
            publishingAccess,
            context,
            hostSession,
            successCallback,
            errorCallback):
        for idx, (entity_ref, traits_data) in enumerate(zip(targetEntityRefs, entityTraitsDatas)):
            # Must provide a reference returned from `preflight`
            if not str(entity_ref).startswith("example://working_ref/"):
                errorCallback(
                    idx, errors.BatchElementError(
                        errors.BatchElementError.ErrorCode.kEntityAccessError,
                        "Cannot publish to this entity"))
                continue

            # Categorise the data to publish
            is_an_unchanged = traits.example.UnchangedTrait.isImbuedTo(traits_data)
            is_an_updated = (traits.example.UpdatedTrait_v2.isImbuedTo(traits_data) or
                             traits.example.UpdatedTrait_v1.isImbuedTo(traits_data))
            is_a_removed = traits.example.RemovedTrait.isImbuedTo(traits_data)
            is_an_added = traits.example.AddedTrait.isImbuedTo(traits_data)

            # Based on the given traits, categorize to pipeline-specific, "type"
            is_an_item = is_an_added and is_an_updated
            is_a_unit = is_a_removed and is_an_updated
            is_an_ingredient = is_an_unchanged and is_an_updated

            # At least and only one, i.e. n-ary xor.
            if int(is_an_item) + int(is_a_unit) + int(is_an_ingredient) != 1:
                errorCallback(
                    idx, errors.BatchElementError(
                        errors.BatchElementError.ErrorCode.kPreflightHintError,
                        "Unsupported traits for publishing to this entity"))
                continue

            v2_updated_trait = traits.example.UpdatedTrait_v2(traits_data)
            v1_updated_trait = traits.example.UpdatedTrait_v1(traits_data)
            if is_an_item:
                if v2_updated_trait.isImbued():
                    foo = v2_updated_trait.getPropertyThatWasRenamed()
                    bar = v2_updated_trait.getPropertyThatWasAdded()
                    do_backend_operation(ref=entity_ref, foo=foo, bar=bar, baz=None)
                else:  # at this point guaranteed that v1 is imbued.
                    foo = v1_updated_trait.getPropertyToRename()
                    baz = v1_updated_trait.getPropertyToRemove()
                    do_backend_operation(ref=entity_ref, foo=foo, bar=None, baz=baz)

                successCallback(idx, EntityReference(f"example://item"))

            elif is_a_unit:
                if v2_updated_trait.isImbued():
                    foo = v2_updated_trait.getPropertyToKeep()
                    do_backend_operation(ref=entity_ref, foo=foo)
                else:  # at this point guaranteed that v1 is imbued.
                    foo = v1_updated_trait.getPropertyToKeep()
                    do_backend_operation(ref=entity_ref, foo=foo)

                successCallback(idx, EntityReference(f"example://unit"))

            elif is_an_ingredient:
                if v2_updated_trait.isImbued():
                    baz = v2_updated_trait.getPropertyThatWasAdded()
                    do_backend_operation(ref=entity_ref, foo=None, baz=baz)
                else:  # at this point guaranteed that v1 is imbued.
                    foo = v1_updated_trait.getPropertyToRemove()
                    do_backend_operation(ref=entity_ref, foo=foo, baz=None)

                successCallback(idx, EntityReference(f"example://ingredient"))


class ExampleEntityReferencePagerInterface(EntityReferencePagerInterface):
    def __init__(self, page_size, results):
        self.__results = results
        self.__idx = 0
        self.__page_size = page_size
        EntityReferencePagerInterface.__init__(self)

    def hasNext(self, _hostSession):
        return self.__idx < len(self.__results)

    def get(self, _hostSession):
        return self.__results[self.__idx:self.__idx + self.__page_size]

    def next(self, _hostSession):
        self.__idx += self.__page_size

    def close(self, _hostSession):
        pass


def do_backend_operation(**kwargs):
    # Do some pipeline-specific backend operation.
    pass


### Host

Next we investigate how a host application might interact with this manager. The following is modified from the generic republish workflow in `generic_republish.ipynb`. Once again, the semantics are nonsense, but hopefully the branching logic is instructive.

In [8]:
from openassetio.hostApi import Manager


# Boilerplate preamble.
manager = Manager(ExampleManagerInterface(), host_session)
context = manager.createContext()
an_entity_ref = manager.createEntityReference(an_entity_ref_str)

# Configure the locale

context.locale.addTraits(v2.specifications.example.ExampleSpecification.kTraitSet)

# The minimum set of traits required to publish to this entity
# reference.
minimum_trait_set = manager.entityTraits(an_entity_ref, access.EntityTraitsAccess.kWrite, context)

# Schema version to use for traits/specifications when creating new
# trait sets/data. Downgrade when manager seems to indicate it doesn't
# support the latest version.
preferred_schema_version = 2

# Whatever the minimum trait set is, we know we're going to publish an
# Example.
desired_trait_set = minimum_trait_set | v2.specifications.example.ExampleSpecification.kTraitSet

# Get the set of traits that have properties the manager can persist.
[policy_for_desired_traits] = manager.managementPolicy(
    [desired_trait_set], access.PolicyAccess.kWrite, context)

# Check if the policy contains a trait that we expect to persist, and
# attempt to fall back to v1 if the trait is not supported. This
# requires us to know that the UpdatedTrait is part of the
# ExampleSpecification (and it is the only trait that carries `resolve`able
# properties).
if not traits.example.UpdatedTrait_v2.isImbuedTo(policy_for_desired_traits):
    # v2 didn't work, try v1.
    desired_trait_set = minimum_trait_set | v1.specifications.example.ExampleSpecification.kTraitSet

    # Get the set of traits that have properties the manager can
    # persist.
    [policy_for_desired_traits] = manager.managementPolicy(
        [desired_trait_set], access.PolicyAccess.kWrite, context)

    if not traits.example.UpdatedTrait_v1.isImbuedTo(policy_for_desired_traits):
        # v1 didn't work either, bail.
        raise Exception(f"Cannot publish an Example to ref {an_entity_ref}")

    # We know the manager doesn't support v2, so downgrade the schema
    # version to use when constructing new trait sets/data. Note that
    # there's no guarantee that the schema version we picked will work
    # for every trait set - for example, the manager may have mixed
    # trait support across schema versions.
    preferred_schema_version = 1

# Filter down the desired traits to only those that are supported.
trait_set_to_publish = desired_trait_set & policy_for_desired_traits.traitSet()

# We want to keep (the minimum amount of) data from the previous
# version, except for the values we're going to provide.
if preferred_schema_version == 2:
    trait_set_to_keep = trait_set_to_publish - v2.specifications.example.ExampleSpecification.kTraitSet
else:
    trait_set_to_keep = trait_set_to_publish - v1.specifications.example.ExampleSpecification.kTraitSet

# Get the properties that we wish to keep from the current version.
data_to_publish = manager.resolve(
    an_entity_ref, trait_set_to_keep, access.ResolveAccess.kRead, context)

# Any traits without properties, or where the manager cannot provide
# them, will be missing from the data. We still need to imbue those
# traits, so that manager knows what kind of entity we are publishing.
data_to_publish.addTraits(minimum_trait_set)

# Get the manager's policy for dictating trait properties, i.e. which
# traits the manager can "drive" for us.
[policy_for_derived_traits] = manager.managementPolicy(
    [trait_set_to_publish], access.PolicyAccess.kManagerDriven, context)

# Check if the manager can derive a value for us.
if traits.example.UpdatedTrait_v2.isImbuedTo(policy_for_derived_traits):
    # Imbue an empty trait, so that the manager is aware in `preflight`
    # that we intend to publish this trait. We will ask the manager to
    # fill in the value for us before calling `register`.
    traits.example.UpdatedTrait_v2.imbueTo(data_to_publish)
elif traits.example.UpdatedTrait_v1.isImbuedTo(policy_for_derived_traits):
    # Fall back to v1.
    traits.example.UpdatedTrait_v1.imbueTo(data_to_publish)
else:
    # If the manager doesn't want to provide a value for entities of
    # this type, use a default.
    if preferred_schema_version == 2:
        traits.example.UpdatedTrait_v2(data_to_publish).setPropertyThatWasRenamed(True)
    else:
        traits.example.UpdatedTrait_v1(data_to_publish).setPropertyToRename(True)

# We can now successfully begin the publishing process.
working_ref = manager.preflight(
    an_entity_ref, data_to_publish, access.PublishingAccess.kWrite, context)

# Check if the manager can provide a value to us.
# First try v2.
if traits.example.UpdatedTrait_v2.kId in policy_for_derived_traits.traitSet():
    derived_data = manager.resolve(
        working_ref, {traits.example.UpdatedTrait_v2.kId}, access.ResolveAccess.kManagerDriven,
        context)

    traits.example.UpdatedTrait_v2(data_to_publish).setPropertyThatWasRenamed(
        traits.example.UpdatedTrait_v2(derived_data).getPropertyThatWasRenamed())

# Fall back to v1.
elif traits.example.UpdatedTrait_v1.kId in policy_for_derived_traits.traitSet():
    derived_data = manager.resolve(
        working_ref, {traits.example.UpdatedTrait_v1.kId}, access.ResolveAccess.kManagerDriven,
        context)

    traits.example.UpdatedTrait_v1(data_to_publish).setPropertyToRename(
        traits.example.UpdatedTrait_v1(derived_data).getPropertyToRename())

# [Do some work to write the new file...]

# We can now finally publish
updated_ref = manager.register(
    working_ref, data_to_publish, access.PublishingAccess.kWrite, context)

## Conclusion

The above explorations have shown that working with versioned traits is entirely possible using the existing API.

There are a few cases where the version of a trait is unimportant, i.e. where the properties are not required since we only wish to use the traits to categorize an entity/relationship/policy/locale. Those cases may warrant API changes to reduce boilerplate. In particular, Specification view classes are currently unsuitable for this use-case. However, this boilerplate does not _prevent_ workflows. Specifications can be used as documentation to help authors construct their own trait set detection logic. So the addition of utility functions is not critical to working with versioned traits.

Conversely, when constructing a trait set or data, a schema version must be chosen by the host/manager. In the case of hosts adapting to managers, the `managementPolicy` API method provides a negotiation mechanism. Otherwise, the host/manager must simply choose a preferred schema version. It is in these circumstances, where new trait data is being created, that Specification view classes are particularly useful.

It seems clear that dealing with mixed trait versions adds a lot of branching logic that could be hard to follow and maintain. A tempting solution is to expose the preferred schema version as a queryable value, so that branching can be performed at a higher level. However, as discussed in [DR023](https://github.com/OpenAssetIO/OpenAssetIO/blob/main/doc/decisions/DR023-Versioning-traits-and-specifications-method.md), this precludes many important workflows, since the ultimate provenance of trait data is unknown in the general case. For example, the manager may combine old data from a database with newly generated data; or the host may load an old project file holding trait data of a previous schema version; or multiple components of a system, each working with a different schema version, may collaborate to produce a trait set/data.